In [1]:
import xarray as xr
import bcdp
import pandas as pd

In [2]:
# Dataset Options
variable_id = 'tos'
experiment_id = ['historical', 'ssp126', 'ssp585']
table_id = 'Omon'
member_id = 'r1i1p1f1'
grid_label = 'gr'
source_id = 'GFDL-ESM4'

# Processing Options
backend = 'scipy'
method = 'linear'
res = (1.0, 1.0)
lat_bnds = (26, 43)
lon_bnds = (-130, -100)
time_bnds = ('1900-01-01', '2101-01-01')

In [ ]:
catfile = 'https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json'
query = dict(variable_id=variable_id, experiment_id=experiment_id, 
             table_id=table_id, member_id=member_id, grid_label=grid_label, 
             source_id=source_id)
zarr_kwargs = dict(storage_options=dict(anon=True))
datasets = bcdp.load_intake_esm(query, catfile=catfile, **zarr_kwargs)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [ ]:
bnds = bcdp.Bounds(time_bnds=('1900-01-01', '2101-01-01'), lat_bnds=lat_bnds, lon_bnds=lon_bnds)
datasets = datasets.subset(bnds).compute()

In [ ]:
grid = bcdp.utils.grid_from_res(res, bnds)
regridded = datasets.regrid(grid, backend=backend, method=method)

In [ ]:
regridded.names

In [ ]:
results = {}
results['ssp126'] = xr.concat([regridded['CMIP.NOAA-GFDL.GFDL-ESM4.historical.Omon.gr'], 
                               regridded['ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp126.Omon.gr']], 'time')
results['ssp585'] = xr.concat([regridded['CMIP.NOAA-GFDL.GFDL-ESM4.historical.Omon.gr'], 
                               regridded['ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.Omon.gr']], 'time')
for scen, ds in results.items():
    ds.name = 'tos'
    fname = f'{variable_id}_{table_id}_{source_id}_{scen}_{member_id}_gr_19000101-21001231.nc'
    ds = ds.dropna(how='all', dim='x').dropna(how='all', dim='y')
    ds.to_netcdf(f'/home/jovyan/shared/data/ecopro/KelpForest/cmip6/{fname}')